# SyftBox Notifications E2E - CLI Daemon Mode

This notebook demonstrates the notification system with the daemon running **completely separately** via CLI.

**Key difference from `notifications_e2e.ipynb`:**
- No `NotificationMonitor` import in this notebook
- Daemon runs in a separate terminal via `syft-notify`
- Proves true background/standalone operation

## Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│  TERMINAL 1: syft-notify daemon                                │
│                                                                 │
│  $ syft-notify                                                  │
│  🔔 Starting notification daemon...                            │
│     DO: test1@openmined.org                                    │
│     Interval: 30s                                               │
│  🔔 JobMonitor started                                         │
│  🔔 PeerMonitor started                                        │
│                        ▲                                        │
│                        │ Polls Drive                            │
└────────────────────────│────────────────────────────────────────┘
                         │
┌────────────────────────│────────────────────────────────────────┐
│  THIS NOTEBOOK         │                                        │
│                        │                                        │
│  DS submits job ───────┼──────> Google Drive                    │
│  DS adds peer ─────────┼──────> Google Drive                    │
│                        │                                        │
│  (No notification imports needed!)                              │
└─────────────────────────────────────────────────────────────────┘
```

---

## Step 0: Start the Notification Daemon (in separate terminal)

**IMPORTANT:** This notebook is designed to work with the CLI daemon (`syft-notify`). 
Do NOT run `NotificationMonitor.start()` from a notebook while the daemon is running - this will cause duplicate notifications!

**Before running this notebook**, open a new terminal and run:

```bash
# First time only - run setup
uv run syft-notify --init

# Then start the daemon
uv run syft-notify
```

You should see:
```
🔔 Starting notification daemon...
   DO: test1@openmined.org
   SyftBox: /home/azureuser/SyftBox_test1@openmined.org
   Interval: 30s
   Press Ctrl+C to stop

🔔 JobMonitor started
🔔 PeerMonitor started
```

**Keep that terminal open** - the daemon will detect events as you run this notebook.

---

## 1. Reset Notification State (Optional)

Run this cell to clear the notification state for a fresh E2E demo.
This ensures peer and job notifications will be sent even if you've run this before.

In [ ]:
from pathlib import Path
import json

state_path = Path.home() / ".syft-creds" / "notification_state.json"

if state_path.exists():
    # Load current state
    with open(state_path) as f:
        state = json.load(f)
    
    # Clear peer snapshots so new peer notifications will fire
    cleared = []
    if "peer_snapshot" in state:
        del state["peer_snapshot"]
        cleared.append("peer_snapshot")
    if "peer_grants_snapshot" in state:
        del state["peer_grants_snapshot"]
        cleared.append("peer_grants_snapshot")
    
    # Clear peer-related notification keys
    jobs = state.get("notified_jobs", {})
    peer_keys = [k for k in list(jobs.keys()) if k.startswith("peer_")]
    for key in peer_keys:
        del jobs[key]
        cleared.append(key)
    
    # Save cleaned state
    with open(state_path, "w") as f:
        json.dump(state, f, indent=2)
    
    print("✅ Cleared peer state:")
    for item in cleared:
        print(f"   - {item}")
    print()
    print("   The daemon will detect peers as NEW on next check cycle")
else:
    print("ℹ️  No notification state file exists yet (first run)")

print()
print("📝 Note: No daemon restart needed - state is reloaded from disk on each check")

## 2. Imports and Configuration

Note: **No notification imports!** The daemon runs separately.

In [ ]:
import syft_client as sc
import os
import json
import uuid
import shutil
from pathlib import Path

# Configure emails
email_do = os.environ.get("SYFT_EMAIL_DO", "test1@openmined.org")
email_ds = os.environ.get("SYFT_EMAIL_DS", "test2@openmined.org")

# Token paths
token_path_do = sc.CREDENTIALS_DIR / "token_do.json"
token_path_ds = sc.CREDENTIALS_DIR / "token_ds.json"

print(f"DO: {email_do}")
print(f"DS: {email_ds}")
print(f"DO token: {token_path_do} - {'✅' if token_path_do.exists() else '❌'}")
print(f"DS token: {token_path_ds} - {'✅' if token_path_ds.exists() else '❌'}")
print()
print("⚠️  Make sure syft-notify is running in another terminal!")

## 3. Login as DO and DS

In [ ]:
# Login as Data Owner
client_do = sc.login_do(
    email=email_do,
    token_path=token_path_do,
)
print(f"✅ DO logged in: {client_do.email}")

In [ ]:
# Login as Data Scientist
client_ds = sc.login_ds(
    email=email_ds,
    token_path=token_path_ds,
)
print(f"✅ DS logged in: {client_ds.email}")

---

## 4. DS Adds DO as Peer

**Watch the daemon terminal!** You should see:
```
📧 Notifying DO (test1@openmined.org) about new peer request from test2@openmined.org
📧 Notifying DS (test2@openmined.org) that peer request was received by test1@openmined.org
```

In [ ]:
# DS adds DO as peer
client_ds.add_peer(email_do)
print(f"✅ DS added {email_do} as peer")
print()
print("👀 Check the daemon terminal for peer notifications!")
print("   (May take up to 30 seconds depending on interval)")

## 5. DO Accepts Peer Request

When DO accepts the peer request, we need to manually trigger the notification since the CLI daemon cannot detect this event automatically.

In [ ]:
# DO adds DS back (accepts peer request)
client_do.add_peer(email_ds)
print(f"✅ DO added {email_ds} as peer")

# Send peer grant notification manually
# (The CLI daemon can't detect this since no new folders are created)
from syft_client.notifications import NotificationMonitor

monitor = NotificationMonitor.from_client(client_do)
monitor.notify_peer_granted(email_ds)
print()
print("👀 Check your email - DS should receive peer grant notification!")

---

## 6. Create Dataset (DO)

In [ ]:
# Download sample dataset
MOCK_DATASET_URL = "https://raw.githubusercontent.com/OpenMined/datasets/refs/heads/main/beach/sales-dataset/mock/sales.csv"
PRIVATE_DATASET_URL = "https://raw.githubusercontent.com/OpenMined/datasets/refs/heads/main/beach/sales-dataset/private/sales.csv"

!curl -s -o /tmp/sales_mock.csv $MOCK_DATASET_URL
!curl -s -o /tmp/sales_private.csv $PRIVATE_DATASET_URL
!echo "# Sales Dataset" > /tmp/readme.md

print("✅ Dataset files downloaded")

In [ ]:
# Create dataset
dataset_name = "sales-data"

# Clean up if exists
for path in [
    client_do.syftbox_folder / client_do.email / "public" / "syft_datasets" / dataset_name,
    client_do.syftbox_folder / "private" / "syft_datasets" / dataset_name,
]:
    if path.exists():
        shutil.rmtree(path)

client_do.create_dataset(
    name=dataset_name,
    mock_path="/tmp/sales_mock.csv",
    private_path="/tmp/sales_private.csv",
    summary="Sales data for analysis",
    readme_path="/tmp/readme.md",
    tags=["sales", "test"],
)
print("✅ Dataset created")

# Sync to Drive
client_do.sync()
print("✅ DO synced")

In [ ]:
# DS syncs to see dataset
client_ds.sync()
print("✅ DS synced")
print(f"   Available datasets: {len(client_ds.datasets.get_all())}")

---

## 7. DS Submits Job

**Watch the daemon terminal!** You should see:
```
📧 Notifying DO (test1@openmined.org) about new job: sales_analysis_XXXX from test2@openmined.org
```

In [ ]:
# Create job code
job_code = '''
import os
import json
import syft_client as sc

# Access private data
data_path = "syft://private/syft_datasets/sales-data/sales_private.csv"
resolved_path = sc.resolve_path(data_path)

with open(resolved_path, "r") as f:
    data = f.read()

# Count lines
line_count = len(data.strip().split("\\n"))

# Save result
os.mkdir("outputs")
with open("outputs/result.json", "w") as f:
    json.dump({"line_count": line_count, "status": "success"}, f)
'''

with open("/tmp/analyze_sales.py", "w") as f:
    f.write(job_code)

print("✅ Job code written")

In [ ]:
# Submit job with unique name
job_name = f"sales_analysis_{uuid.uuid4().hex[:8]}"

client_ds.submit_python_job(
    user=email_do,
    code_path="/tmp/analyze_sales.py",
    job_name=job_name,
)
print(f"✅ Job '{job_name}' submitted to {email_do}")
print()
print("👀 Check the daemon terminal for job notification!")
print("   (May take up to 30 seconds depending on interval)")

---

## 8. DO Syncs, Approves, and Executes Job

**Watch the daemon terminal!** After approval and execution, you should see:
```
📧 Job approved notification sent to test2@openmined.org
📧 Job executed notification sent to test2@openmined.org
```

In [ ]:
# DO syncs to receive the job
client_do.sync()
print(f"✅ DO synced - Jobs received: {len(client_do.job_client.jobs)}")

In [ ]:
# Get the LATEST job (by submission time) that's in inbox status
pending_jobs = [j for j in client_do.job_client.jobs if j.status == "inbox"]
if not pending_jobs:
    print("❌ No pending jobs found. The job may have already been processed.")
    print("   Available jobs:", [j.name for j in client_do.job_client.jobs])
else:
    job = pending_jobs[0]  # Most recent pending job
    print(f"Job: {job.name}")
    print(f"Submitted by: {job.submitted_by}")
    print(f"Status: {job.status}")

    job.approve()
    print(f"\n✅ Job '{job.name}' approved")
    print()
    print("👀 Check daemon terminal for 'approved' notification!")

In [ ]:
# Execute the job
client_do.job_runner.process_approved_jobs()
print("✅ Job executed")
print()
print("👀 Check daemon terminal for 'executed' notification!")

# Sync results back
client_do.sync()
print("✅ DO synced results")

---

## 9. DS Receives Results

In [ ]:
# DS syncs to get results
client_ds.sync()
print("✅ DS synced")

# Check results
ds_job = client_ds.job_client.jobs[-1]
print(f"\nJob: {ds_job.name}")
print(f"Output paths: {ds_job.output_paths}")

if ds_job.output_paths:
    with open(ds_job.output_paths[0], "r") as f:
        result = json.load(f)
    print(f"\n📊 Result: {result}")

---

## 10. Verify Notifications Were Sent

Check the notification state file to see what was sent:

In [ ]:
# Check notification state (created by the daemon)
state_path = Path.home() / ".syft-creds" / "notification_state.json"

if state_path.exists():
    with open(state_path) as f:
        state = json.load(f)
    
    print("📬 Notifications sent by daemon:")
    print("=" * 50)
    
    # Show job notifications
    jobs = state.get("notified_jobs", {})
    for key, events in jobs.items():
        if not key.startswith("msg_") and not key.startswith("peer_"):
            print(f"\n📋 {key}:")
            for event in events:
                print(f"   ✉️  {event}")
    
    # Show peer notifications
    peer_keys = [k for k in jobs.keys() if k.startswith("peer_")]
    if peer_keys:
        print(f"\n👥 Peer notifications:")
        for key in peer_keys:
            print(f"   ✉️  {key}: {jobs[key]}")
else:
    print("❌ No notification state file found.")
    print("   Is the daemon running?")

---

## Summary

### What the CLI Daemon Can Detect

| Event | Auto-Detected | Notification |
|-------|---------------|--------------|
| DS adds DO as peer | ✅ Yes | Email to DO + DS |
| DO accepts peer request | ❌ No* | Manual call in notebook |
| New job submitted | ✅ Yes | Email to DO |
| Job approved | ✅ Yes | Email to DS |
| Job executed | ✅ Yes | Email to DS |

*DO's `add_peer()` doesn't create new folders, so the daemon can't detect it. We call `monitor.notify_peer_granted()` manually.

### Emails Sent

**To DO (test1@openmined.org):**
- New Peer Request from DS (daemon)
- New Job submitted (daemon)

**To DS (test2@openmined.org):**
- Peer Request Sent confirmation (daemon)
- Peer Request Accepted (manual call)
- Job Approved (daemon)
- Job Completed (daemon)

### Stop the Daemon

Go to the terminal running `syft-notify` and press `Ctrl+C`.

---

## Troubleshooting

### Daemon not detecting events?

1. Check the daemon is running (`syft-notify`)
2. Wait for the polling interval (default 30s)
3. Check the config has correct `do_email` matching this notebook

### No emails received?

1. Check spam folder
2. Verify Gmail token is valid: `cat ~/.syft-creds/gmail_token.json`
3. Check notification_state.json to see if notifications were marked as sent

### Reset and try again?

```bash
# Clear notification state to re-send
rm ~/.syft-creds/notification_state.json

# Restart daemon
syft-notify
```